In [8]:
import gensim
from elasticsearch import Elasticsearch
from gensim.models import KeyedVectors
from sklearn.datasets import fetch_20newsgroups

# Load the dataset
newsgroups = fetch_20newsgroups(subset='all')

# Preprocess the documents
preprocessed_docs = []
for doc in newsgroups.data:
    # Tokenize the document
    tokens = gensim.utils.simple_preprocess(doc.lower())
    # Remove stop words and stem the tokens
    stemmed_tokens = [gensim.parsing.porter.PorterStemmer().stem(token) for token in tokens if token not in gensim.parsing.preprocessing.STOPWORDS]
    # Join the stemmed tokens back into a string
    preprocessed_doc = ' '.join(stemmed_tokens)
    preprocessed_docs.append(preprocessed_doc)

#  model
model = KeyedVectors.load_word2vec_format('C:/Users/Dell/Downloads/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)


# Initialize Elasticsearch client with URL
es = Elasticsearch("https://localhost:9200/", ca_certs="C:\elastic stack\elasticsearch-8.10.4-windows-x86_64\elasticsearch-8.10.4\config\certs\http_ca.crt", basic_auth=("elastic", "sTyuyUy3Usw7igqaIJ0j"))


# Delete the index if it already exists
index_name = 'vec'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Create index with appropriate mappings
index_mappings = {
    'mappings': {
        'properties': {
            'text': {
                'type': 'text'
            },
            'vector': {
                'type': 'dense_vector',
                'dims': 300
            }
        }
    }
}
es.indices.create(index=index_name, body=index_mappings)

# Iterate over preprocessed documents and generate vectors
for i, doc in enumerate(preprocessed_docs):
    # Split the preprocessed document into tokens
    tokens = doc.split()
    # Generate the vector for the document by averaging the vectors of its tokens
    vector_sum = 0
    count = 0
    for token in tokens:
        if token in model.wv:
            vector_sum += model.wv[token]
            count += 1
    if count > 0:
        vector = vector_sum / count
        # Store the document and its vector in the Elasticsearch index
        es.index(index=index_name, id=i, body={'text': doc, 'vector': vector.tolist()})

ConnectionTimeout: Connection timed out

In [6]:
import gensim
from elasticsearch import Elasticsearch

# Initialize Elasticsearch client with URL
es = Elasticsearch("https://localhost:9200/", ca_certs="C:\elastic stack\elasticsearch-8.10.4-windows-x86_64\elasticsearch-8.10.4\config\certs\http_ca.crt", basic_auth=("elastic", "sTyuyUy3Usw7igqaIJ0j"))


# # Load the pre-trained FastText model
# model = gensim.models.FastText.load('"C:/Users/sudha/Downloads/model.bin.wv.vectors_ngrams.npy"')

# Get user query
user_query = input('Enter your query: ')

# Preprocess the user query
tokens = gensim.utils.simple_preprocess(user_query.lower())
stemmed_tokens = [gensim.parsing.porter.PorterStemmer().stem(token) for token in tokens if token not in gensim.parsing.preprocessing.STOPWORDS]
preprocessed_query = ' '.join(stemmed_tokens)

# Generate the vector for the user query by averaging the vectors of its tokens
vector_sum = 0
count = 0
for token in preprocessed_query.split():
    if token in model.wv:
        vector_sum += model.wv[token]
        count += 1
if count > 0:
    query_vector = vector_sum / count
    print(f'Query vector: {query_vector}')

    # Search for similar documents using Elasticsearch
    search_body = {
        'query': {
            'script_score': {
                'query': {
                    'match_all': {}
                },
                'script': {
                    'source': 'cosineSimilarity(params.query_vector, "vector") + 1.0',
                    'params': {
                        'query_vector': query_vector.tolist()
                    }
                }
            }
        },
        '_source': {
            'includes': ['text']
        }
    }
    search_results = es.search(index='my_index', body=search_body)['hits']['hits']

    # Print the top 10 most similar documents
    print('Top 10 most similar documents:')
    for i, hit in enumerate(search_results[:10]):
        print(f'{i+1}. {hit["_source"]["text"]}')
else:
    print('No valid tokens in query')

Query vector: [-0.04956055 -0.11083984  0.03320312  0.3984375  -0.25195312 -0.02685547
 -0.37695312  0.078125    0.48828125  0.06640625  0.37695312  0.10351562
 -0.07421875  0.30859375 -0.2734375   0.11816406 -0.14941406  0.26757812
 -0.02368164 -0.00836182  0.05175781 -0.0378418  -0.06347656 -0.10253906
  0.38476562  0.1953125   0.20117188 -0.31445312  0.30078125  0.03417969
  0.5         0.04614258 -0.05834961  0.31445312 -0.4375     -0.03540039
 -0.28515625  0.17578125  0.16796875  0.11474609  0.07324219  0.07568359
  0.13867188 -0.05957031  0.05126953 -0.02807617  0.10791016 -0.12353516
 -0.3203125   0.22167969 -0.03857422 -0.03063965 -0.01269531  0.05273438
  0.14550781 -0.16308594  0.02038574 -0.02197266 -0.09130859 -0.10498047
 -0.06591797  0.13574219  0.21582031 -0.03515625 -0.03393555 -0.14257812
 -0.12988281 -0.15039062 -0.05175781 -0.07714844 -0.00805664  0.12402344
  0.21191406 -0.04370117 -0.328125   -0.36523438  0.25976562 -0.14648438
  0.06689453 -0.14941406  0.04541016 

C:\Users\Dell\AppData\Local\Temp\ipykernel_16648\3344487088.py:23: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if token in model.wv:
C:\Users\Dell\AppData\Local\Temp\ipykernel_16648\3344487088.py:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector_sum += model.wv[token]


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [my_index]', my_index, index_or_alias)

In [10]:
from elasticsearch import Elasticsearch
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, ndcg_score
import numpy as np

# Initialize Elasticsearch client with URL
es = Elasticsearch("https://localhost:9200/", ca_certs="C:\elastic stack\elasticsearch-8.10.4-windows-x86_64\elasticsearch-8.10.4\config\certs\http_ca.crt", basic_auth=("elastic", "sTyuyUy3Usw7igqaIJ0j"))

# Load the dataset
newsgroups = fetch_20newsgroups(subset='all')

# Preprocess the documents
preprocessed_docs = []
for doc in newsgroups.data:
    # Tokenize the document
    tokens = gensim.utils.simple_preprocess(doc.lower())
    # Remove stop words and stem the tokens
    stemmed_tokens = [gensim.parsing.porter.PorterStemmer().stem(token) for token in tokens if token not in gensim.parsing.preprocessing.STOPWORDS]
    # Join the stemmed tokens back into a string
    preprocessed_doc = ' '.join(stemmed_tokens)
    preprocessed_docs.append(preprocessed_doc)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_docs, newsgroups.target, test_size=0.2, random_state=42)

# Delete the index if it already exists
index_name = 'vector_retrieval'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Create index with appropriate mappings
index_mappings = {
    'mappings': {
        'properties': {
            'text': {
                'type': 'text'
            },
            'vector': {
                'type': 'dense_vector',
                'dims': 300
            }
        }
    }
}
es.indices.create(index=index_name, body=index_mappings)

# Iterate over preprocessed documents and generate vectors
for i, doc in enumerate(X_train):
    # Split the preprocessed document into tokens
    tokens = doc.split()
    # Generate the vector for the document by averaging the vectors of its tokens
    vector_sum = 0
    count = 0
    for token in tokens:
        if token in model.wv:
            vector_sum += model.wv[token]
            count += 1
    if count > 0:
        vector = vector_sum / count
        # Store the document and its vector in the Elasticsearch index
        es.index(index=index_name, id=i, body={'text': doc, 'vector': vector.tolist()})

# Define the benchmark queries and their relevance judgments
queries = [
    'computer graphics',
    'baseball',
    'Christianity',
    'guns',
    'medical treatment',
    'NASA',
    'Windows vs. Macintosh',
    'cars',
    'politics',
    'cars'
]
relevance_judgments = [
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
]

# Search for each query and evaluate the performance
for i, query in enumerate(queries):
    # Preprocess the query
    tokens = gensim.utils.simple_preprocess(query.lower())
    stemmed_tokens = [gensim.parsing.porter.PorterStemmer().stem(token) for token in tokens if token not in gensim.parsing.preprocessing.STOPWORDS]
    preprocessed_query = ' '.join(stemmed_tokens)

    # Generate the vector for the query by averaging the vectors of its tokens
    vector_sum = 0
    count = 0
    for token in preprocessed_query.split():
        if token in model.wv:
            vector_sum += model.wv[token]
            count += 1
    if count > 0:
        query_vector = vector_sum / count

        # Search for similar documents using Elasticsearch
        search_body = {
            
            'query': {
                'script_score': {
                    'query': {
                        'match_all': {}
                    },
                    'script': {
                        'source': 'cosineSimilarity(params.query_vector, "vector") + 1.0',
                        'params': {
                            'query_vector': query_vector.tolist()
                        }
                    }
                }
            },
            '_source': {
                'includes': ['text']
            }
        }
        search_results = es.search(index=index_name, body=search_body)['hits']['hits']

        # Evaluate the performance using standard IR metrics
        y_true = np.zeros(len(search_results))
        # Evaluate the performance using standard IR metrics
y_true = np.zeros(len(search_results))
for relevance_index in relevance_judgments[i]:
    if relevance_index < len(y_true):
        y_true[relevance_index] = 1
    else:
        print(f"Warning: Index {relevance_index} is out of bounds.")

        
        y_pred = np.zeros(len(search_results))
        for j, hit in enumerate(search_results):
            if j in relevance_judgments[i]:
                y_pred[j] = 1
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        ap = average_precision_score(y_true, y_pred)
        ndcg = ndcg_score([y_true], [y_pred])

        # Print the results
        print(f'Query: {query}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1: {f1:.4f}')
        print(f'MAP: {ap:.4f}')
        print(f'NDCG: {ndcg:.4f}')
        print()

ConnectionTimeout: Connection timed out